In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager

import numpy as np
from numpy import random
from time import sleep

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.nytimes.com/games/wordle/index.html')

actions = ActionChains(driver)
actions.click()
actions.perform()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Benjamin\.wdm\drivers\chromedriver\win32\100.0.4896.60]
C:\Users\Benjamin\AppData\Local\Temp\ipykernel_5860\4029482767.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
wordleGuesses = np.loadtxt('wordle-guesses.txt', dtype = str)

def make_random_guess():
    guessIdx = np.random.randint(low = 0, high = len(wordleGuesses))
    guess = wordleGuesses[guessIdx]
    actions.send_keys(guess)
    actions.send_keys(Keys.RETURN)
    actions.perform()

def evaluate_random_guess(idx):
    """
    
    
    """

    game_app = driver.find_element(By.TAG_NAME , 'game-app')
    game_rows = driver.execute_script("return arguments[0].shadowRoot.getElementById('board')", game_app).find_elements(By.TAG_NAME, 'game-row')
    letters = driver.execute_script('return arguments[0].shadowRoot', game_rows[idx]).find_elements(By.CSS_SELECTOR , 'game-tile')

    # Quantize evaluation 
    eval_to_int = {
        'correct' : 2,
        'present' : 1,
        'absent'  : 0
    }

    # Evaluate guess
    correctness = 0
    for letter in letters:
        correctness += eval_to_int[letter.get_attribute('evaluation')]
    correctness /= 10
    print('Correctness: {:.2f}'.format(correctness))

In [ ]:
for i in np.arange(6):
    make_random_guess()
    evaluate_random_guess(i)
    sleep(2.5)

In [4]:
wordleGuesses = np.loadtxt('wordle-guesses.txt', dtype = str)
# Subset small sample of words
subsetGuesses = np.random.choice(wordleGuesses, size = 100, replace = False)

In [27]:
# Interpret gameboard
game_app = driver.find_element(By.TAG_NAME , 'game-app')
game_rows = driver.execute_script("return arguments[0].shadowRoot.getElementById('board')", game_app).find_elements(By.TAG_NAME, 'game-row')
game_tiles = driver.execute_script('return arguments[0].shadowRoot', game_rows[1]).find_elements(By.CSS_SELECTOR , 'game-tile')

In [34]:
new_state = []
word = 'shank'
# Parse pattern
for i, tile in enumerate(game_tiles):
  eval = tile.get_attribute('evaluation')
  letter = tile.get_attribute('letter')
  if eval == 'correct':
    # Add words to new state with letter at position `i` in word
    # new_state += [word for word in subsetGuesses if word[i] == letter]
    pass
  elif eval == 'present':
    # Add words from state with letter in word
    # new_state += [word for word in subsetGuesses if letter in word]
    pass
  else:
    # Add words from state without letter is in word
    new_state += [word for word in subsetGuesses if letter not in word ]
    pass
new_state = np.unique(new_state)